# Bonus Tutorial: Optimizing a Simple Solar Home System

This bonus tutorial demonstrates how PyPSA can model a small, real-life energy system, like a Solar Home System (SHS) commonly used in off-grid areas or for backup power. This is relevant for DIY enthusiasts or understanding small-scale energy management in contexts like Kenya.

**Scenario:** We have a small household with basic electricity needs (lights, phone charging). They have installed a small solar PV panel and a battery.

**Goal:** We want to simulate the operation of this system over a typical day to see how the solar panel charges the battery and powers the load. This helps assess if the chosen component sizes (PV panel, battery) are likely sufficient for the expected daily usage, minimizing reliance on any potential backup (or minimizing unmet load if fully off-grid).

**Concepts Covered:**
- Defining `Bus`, `Generator` (PV), `Store` (Battery), `Load`.
- Using simple time-series data for solar availability (`p_max_pu`) and load (`p_set`).
- Running a basic operational simulation (`optimize`).
- Visualizing system behaviour (PV output, Load, Battery State of Charge).

In [ ]:
import pypsa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Create a network for a single day (24 hours)
network = pypsa.Network()
snapshots = pd.date_range("2024-01-01 00:00", periods=24, freq="h")
network.set_snapshots(snapshots)

# Add the main electricity bus for the home
network.add("Bus", "Household AC Bus", carrier="AC")

# Add Carriers (optional, but good practice)
network.add("Carrier", "Solar")
# We could add a 'Grid' or 'Kerosene' carrier if modeling backup, but let's keep it simple (off-grid)
# network.add("Carrier", "Grid")

## Define System Components

Let's define the PV Panel, Battery, and Household Load.

In [ ]:
# 1. Solar PV Panel (Generator)
# Small panel, e.g., 150W peak
pv_capacity_kw = 0.15 # kW
# Simple sunny day profile (0 at night, peaks at midday)
pv_profile = pd.Series([0, 0, 0, 0, 0, 0.1, 0.3, 0.6, 0.8, 0.95, 1.0, 0.95, 
                       0.8, 0.6, 0.3, 0.1, 0, 0, 0, 0, 0, 0, 0, 0], index=network.snapshots)
network.add(
    "Generator",
    "PV Panel",
    bus="Household AC Bus",
    p_nom=pv_capacity_kw, # Use kW for smaller scale
    p_nom_extendable=False,
    carrier="Solar",
    marginal_cost=0, # Assume zero cost for sunlight
    p_max_pu=pv_profile # The availability profile
)

# 2. Battery Storage (Store)
# Small battery, e.g., 100Ah @ 12V = 1.2 kWh. Let's use kWh.
battery_capacity_kwh = 1.2 # kWh
battery_power_kw = 0.5 # Max charge/discharge rate (kW)
network.add(
    "Store",
    "Home Battery",
    bus="Household AC Bus",
    e_nom=battery_capacity_kwh, # Energy capacity in kWh
    p_nom=battery_power_kw,     # Power capacity in kW
    e_nom_extendable=False,
    p_nom_extendable=False,
    e_cyclic=True, # Assume battery can cycle daily
    e_initial=0.5, # Start at 50% state of charge
    e_min_pu=0.1,  # Min state of charge (10%)
    e_max_pu=0.95, # Max state of charge (95%)
    efficiency_store=0.9, # Charging efficiency
    efficiency_dispatch=0.9, # Discharging efficiency
    marginal_cost=0.01 # Very small cost to encourage using battery when needed
)

# 3. Household Load (Load)
# Basic load: lights in evening, phone charging during day/evening
# Let's define load in kW
load_profile_kw = pd.Series([0.01]*6 + [0.02]*2 + [0.05]*4 + [0.02]*4 + [0.1]*4 + [0.05]*4, index=network.snapshots)
network.add(
    "Load",
    "Household Load",
    bus="Household AC Bus",
    p_set=load_profile_kw # The demand profile in kW
)

# (Optional) Add a very expensive 'unmet load' generator if purely off-grid
# This represents load shedding. It acts as a backup sink for energy if the optimization
# problem becomes infeasible, ensuring it can always find *a* solution.
# Its high cost ensures it's only used as a last resort.
network.add("Generator", "Unmet Load", bus="Household AC Bus", p_nom=1, marginal_cost=1000)

## Simulate System Operation

Now, we run the linear optimal power flow (`lopf`) to simulate how the system meets the load using the PV and battery.

In [ ]:
# Solve the operational problem
print("Simulating Solar Home System operation...")
status, condition = network.optimize()

print(f"Solver Status: {status}, Termination Condition: {condition}")

if status == "ok" and condition == "optimal":
    print(f"Simulation successful. Total cost (proxy for unmet load/losses): {network.objective:.2f}")
else:
    print("Simulation did not find an optimal solution. Check model setup or solver.")

## Visualize the Results

Let's plot the key results: PV generation, load demand, battery charging/discharging, and battery state of charge (SOC).

In [ ]:
if status == "ok" and condition == "optimal":
    # Create a figure with subplots
    fig, axes = plt.subplots(3, 1, figsize=(10, 12), sharex=True)

    # 1. Plot PV Generation vs Load
    network.generators_t.p["PV Panel"].plot(ax=axes[0], label="PV Generation (kW)", color='orange')
    network.loads_t.p_set["Household Load"].plot(ax=axes[0], label="Load Demand (kW)", color='blue', linestyle='--')
    axes[0].set_ylabel("Power (kW)")
    axes[0].set_title("PV Generation and Load Demand")
    axes[0].legend()
    axes[0].grid(True)

    # 2. Plot Battery Dispatch (Charge/Discharge)
    # network.stores_t.p shows dispatch (positive) and storage (negative)
    network.stores_t.p["Home Battery"].plot(ax=axes[1], label="Battery Power (kW) [Discharge > 0, Charge < 0]", color='green')
    axes[1].set_ylabel("Power (kW)")
    axes[1].set_title("Battery Charging/Discharging Power")
    axes[1].legend()
    axes[1].grid(True)
    axes[1].axhline(0, color='grey', linewidth=0.5) # Add zero line

    # 3. Plot Battery State of Charge (SOC)
    network.stores_t.e["Home Battery"].plot(ax=axes[2], label="Battery Energy (kWh)", color='purple')
    axes[2].set_ylabel("Energy (kWh)")
    axes[2].set_title("Battery State of Charge")
    axes[2].legend()
    axes[2].grid(True)
    # Add min/max SOC lines based on e_nom and min/max_pu
    min_soc = network.stores.at["Home Battery", "e_min_pu"] * network.stores.at["Home Battery", "e_nom"]
    max_soc = network.stores.at["Home Battery", "e_max_pu"] * network.stores.at["Home Battery", "e_nom"]
    axes[2].axhline(min_soc, color='red', linestyle=':', linewidth=1, label=f'Min SOC ({min_soc:.2f} kWh)')
    axes[2].axhline(max_soc, color='red', linestyle=':', linewidth=1, label=f'Max SOC ({max_soc:.2f} kWh)')
    axes[2].legend()

    # Add unmet load if relevant
    if "Unmet Load" in network.generators_t.p.columns and network.generators_t.p["Unmet Load"].sum() > 0.01:
        unmet = network.generators_t.p["Unmet Load"]
        axes[0].plot(unmet.index, unmet, label="Unmet Load (kW)", color='red', linestyle=':')
        axes[0].legend() # Update legend
        print(f"Warning: Total unmet load was {unmet.sum():.2f} kWh")

    plt.xlabel("Time of Day")
    plt.tight_layout()
    plt.show()
else:
    print("Cannot plot results as simulation was not successful.")

## Summary & Interpretation

This simple example shows how PyPSA can model the interplay between solar generation, battery storage, and household demand.

- **During the day:** The PV panel generates electricity. This power meets the immediate load, and any excess is used to charge the battery (observe negative power in the second plot and rising SOC in the third plot).
- **During the evening/night:** When PV generation is low or zero, the battery discharges (positive power in the second plot, falling SOC in the third plot) to meet the household load.
- **Optimization Goal:** The `optimize` function implicitly minimizes the cost. Since PV is free (marginal_cost=0) and the battery has a tiny cost, the system prioritizes using PV directly, then storing excess PV in the battery, and finally discharging the battery to meet the load. The expensive "Unmet Load" generator is only used if the PV and battery cannot meet the demand.

This model could be extended to:
- Use more realistic, granular weather data and load profiles.
- Include costs for components to perform sizing optimization (like Tutorial 05).
- Model grid connection or other backup sources.
- Analyze different battery sizes or PV panel capacities.